In [1]:
##tickers = ['AMZN', 'TSLA', 'MELI', 'NFLX', 'SHOP', 'SQ'] #GOOG AMZN
tickers = ['TSLA'] # Will use TSLA for now

In [2]:
import os 
import pandas as pd
import json
import requests
import numpy as np
import math
from pathlib import Path
import csv # using Python module
from ast import literal_eval

import matplotlib.pyplot as plt
%matplotlib inline


Bad key text.latex.preview in file /Users/dallolmac/opt/anaconda3/envs/pyvizenv/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 123 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.0/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file /Users/dallolmac/opt/anaconda3/envs/pyvizenv/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 155 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.0/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.jpeg_quality in file /Users/dallolmac/opt/anaconda3/envs/pyvizenv/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 418 ('savefig

In [9]:
def loadStockData(ticker):
    # Read ticker file
    #file_path = (f'./Resources/Data/{ticker}.csv')
    file_path = (f'./Resources/{ticker}.csv')
    df = pd.read_csv(file_path,index_col='timestamp', encoding='utf8')# parse_dates=True, infer_datetime_format=True, )
    df.index = pd.to_datetime(df.index, utc=True).tz_convert(tz="America/New_York")

    return df
    

In [10]:
def saveStockData(ticker, df):
    # Read ticker file
    #file_path = (f'./Resources/Data/{ticker}.csv')
    file_path = (f'./Resources/{ticker}.csv')
    df.to_csv(file_path)

    return True

In [11]:
def loadSenti(ticker):
    # Read sentiment
    file_path = (f'./Resources/{ticker}_news_sent-NEW.csv')
    df = pd.read_csv(file_path, index_col='date')# parse_dates=True, infer_datetime_format=True)
    df.index = pd.to_datetime(df.index, utc=True).tz_convert(tz="America/New_York")
    
    # Drop extra columns
    df = df.drop(['Unnamed: 0', 'news_url', 'image_url', 'topics'], axis=1)
    # Add a column to keep track of which aritcles have already been processed
    df['processed'] = False
    
    # Change negative, neutral and positive ot -1, 0 , 1
    df.loc[df.sentiment == 'Negative', 'sentiment']  = -1    
    df.loc[df.sentiment == 'Neutral', 'sentiment']   = 0
    df.loc[df.sentiment == 'Positive', 'sentiment']  = 1
    
    # Change tickers to a list from string of values
    df.tickers = df.tickers.apply(lambda x: literal_eval(x))    
   

    
    return df

In [12]:
def getSentiScore(dfs, timestamp):
    
    score = 0
    num_tickers_weight = .5
    sentiment_weight   = .5

    
    # Get all articles up to the timestamp that have not already been used before
    dfs['td'] =  (timestamp.date() - dfs.index.date)
    dfs.td    = dfs.td.apply(lambda x: x.days)
    sf = dfs[(dfs.index <= timestamp) & (dfs.processed==False) & ( dfs.td < 2)]
    
    # dro the extra column now
    dfs = dfs.drop('td', axis=1)
    
    # just return if noting came back with a sore of 0
    if len(sf) < 1:
        return score

    # Weight if the news is just about TSLA or among many others
    #num_tickers       = sf.tickers.apply(lambda x: len(x)).sum()
    num_tickers_score = sf.tickers.apply(lambda x: 1/len(x)).sum()
    
    sentiment = sf.sentiment.sum()/sf.sentiment.count()
    
    # Calculat score and multiple by sentiment at the end to keep the negative or positiove
    score = ((num_tickers_score * num_tickers_weight) + (abs(sentiment) * sentiment_weight) ) 
    if sentiment < 0:
        score = score * -1
    
    
    return score

In [13]:
# Process function. Reads ticker file and sentiment file and tries to add 

for ticker in tickers:
    print(f'--> {ticker}')
    
    # Load stock and sentiment data
    df  = loadStockData(ticker)
    dfs = loadSenti(ticker)
    
    # Create a sentiment score, based on relevance of the sentiment 
    # A score 0 - 1 represents the rating based on number of artlces about some news within the same period
    # future improvment... maybe score certain media outlets more than others
    df['senti_score'] = None
    
    # Loop through each date in the data
    dates = df.groupby(by=df.index.date)
    first_day = True
    for one_day_index, one_day in dates:
        print('---> Processing Sentiment for ', one_day.index[0].date())

        # Loop thorugh each minute or row within the date provided
        for one_min_index, one_min in one_day.iterrows():
        
            # Set Senti score
            df.loc[one_min.name, 'senti_score'] = getSentiScore(dfs, one_min.name)
    
    
    # Update Stock CSV and get new ticker
    saveStockData(ticker, df)

    
    

--> TSLA
---> Porcessing Sentiment for  2022-03-01


In [2]:
ticker = 'tsla'
ticker = ticker.upper()
            
        

# Registered for Free Trail to get the below API
news_api= 'keoee7ujrknal5ci2dpkhlrjddwcqqg9dmyecpgj'
date_range = ['03012022-03012022',
            '03022022-0302022',
        ]

df = pd.DataFrame()
print(f'---> Getting {ticker} news...')
for quarter in date_range:
    page_num = 1
    keep_going = True
    while keep_going:
        print(f'     -> Quarter {quarter} - Page {page_num}')
        try:
            #print('A')
            link = requests.get(f'https://stocknewsapi.com/api/v1?tickers={ticker}&items=50&date={quarter}&page={page_num}&token=' + news_api)
            link.content
            news = link.json()
            #print('B')
            df = df.append(pd.json_normalize(news['data']))
            page_num = page_num + 1
                
        except:
            print(f'Error on {ticker} - {quarter} - {page_num}')
            keep_going = False

---> Getting TSLA news...
     -> Quarter 03012022-03012022 - Page 1
Error on TSLA - 03012022-03012022 - 1
     -> Quarter 03022022-0302022 - Page 1
Error on TSLA - 03022022-0302022 - 1
